In [4]:
import xarray as xr
import numpy as np
import dask.array as da
# import tensorflow as tf

In [5]:
data = np.array(
    [
        [[1,2],
         [3,4],
         [5,6]],
        [[1,2],
         [3,4],
         [5,6]]
    ]
)

coords = {'bands': [1,2],'x':[1,2,3],'y':[1,2]}
dims = ('bands','x','y')
array = xr.DataArray(data=data,coords=coords,dims=dims)
array

<xarray.DataArray (bands: 2, x: 3, y: 2)>
array([[[1, 2],
        [3, 4],
        [5, 6]],

       [[1, 2],
        [3, 4],
        [5, 6]]])
Coordinates:
  * bands    (bands) int64 1 2
  * x        (x) int64 1 2 3
  * y        (y) int64 1 2

In [112]:
inv = array[dict(x=2,y=1,bands=slice(0, 2))]
inv

<xarray.DataArray (bands: 2)>
array([6, 6])
Coordinates:
  * bands    (bands) int64 1 2
    x        int64 3
    y        int64 2

In [ ]:
function = lambda *args,**kwags: sum(list(args))
xr.apply_ufunc(function, inv[0], inv[1],inv[2],inv[3],inv[4])

In [97]:
# np.take_along_axis(array,0,axis=0)
array[1,:1]

<xarray.DataArray (x: 1, y: 2)>
array([[1, 2]])
Coordinates:
    bands    int64 2
  * x        (x) int64 1
  * y        (y) int64 1 2

In [71]:
def function(ins):
    print(ins)
    return ins

future = da.apply_along_axis(function, 0, array,dtype=np.double, shape=(2,))
future.compute()

[1 1]
[2 2]
[3 3]
[4 4]
[5 5]
[6 6]


array([[[1, 2],
        [3, 4],
        [5, 6]],

       [[1, 2],
        [3, 4],
        [5, 6]]])

In [6]:
dask_array = array.chunk({'bands': 2})
dask_array.variable.data

dask.array<xarray-<this-array>, shape=(2, 3, 2), dtype=int64, chunksize=(2, 3, 2), chunktype=numpy.ndarray>

In [7]:
function = lambda args,**kwargs: print(args,type(args),len(args)); 1
r = da.apply_along_axis(func1d=function,axis=0,arr=dask_array,dtype=np.int16,shape=(1,))

In [8]:
r.compute()

[1 1] <class 'numpy.ndarray'> 2
[2 2] <class 'numpy.ndarray'> 2
[3 3] <class 'numpy.ndarray'> 2
[4 4] <class 'numpy.ndarray'> 2
[5 5] <class 'numpy.ndarray'> 2
[6 6] <class 'numpy.ndarray'> 2


array([[None, None],
       [None, None],
       [None, None]], dtype=object)

In [115]:
data_1 = xr.open_rasterio('../data/prueba/pequena.tif',chunks={'band':6, 'y':1})
data_1.variable.data

,Array,Chunk
Bytes,327.85 MB,88.92 kB
Shape,"(6, 3687, 3705)","(6, 1, 3705)"
Count,3688 Tasks,3687 Chunks
Type,float32,numpy.ndarray


In [117]:
inv = data_1[dict(band=slice(0, 6))]
inv.values

array([[[ 275.5,  260. ,  218. , ..., 1076. ,  999. ,  950.5],
        [ 325. ,  251. ,  264.5, ..., 1017. , 1002.5,  859. ],
        [ 278. ,  322. ,  262.5, ...,  961. ,  880.5,  801.5],
        ...,
        [ 562. ,  599. ,  553. , ..., 1085.5, 1022. , 1139. ],
        [ 541. ,  564. ,  568. , ...,  879.5,  782. ,  803. ],
        [ 495. ,  509. ,  566. , ...,  973. ,  869. ,  793. ]],

       [[ 181. ,  175.5,  155. , ..., 2120.5, 2009. , 1928. ],
        [ 205. ,  168.5,  178.5, ..., 2081. , 2016.5, 1807. ],
        [ 181. ,  185. ,  176. , ..., 2096. , 1915. , 1777. ],
        ...,
        [ 815. ,  884. ,  815. , ..., 1032. , 1271. , 2090.5],
        [ 807. ,  855. ,  866. , ..., 1507. , 1274. , 1364. ],
        [ 699. ,  782. ,  852. , ..., 1195. , 1026. , 1370. ]],

       [[ 213.5,  196.5,  153. , ..., 2378. , 2290. , 2236. ],
        [ 251. ,  196.5,  194.5, ..., 2366. , 2334.5, 2140. ],
        [ 204. ,  245. ,  195. , ..., 2306. , 2188.5, 2111.5],
        ...,
        [394

In [122]:
# function = lambda *args,**kwags: 
# xr.apply_ufunc(function, inv[0], inv[1],inv[2],inv[3],inv[4],inv[5])
a = inv.values[0][:2,:2]
b = inv.values[1][:2,:2]
c = inv.values[2][:2,:2]
d = inv.values[3][:2,:2]
e = inv.values[4][:2,:2]
f = inv.values[5][:2,:2]

In [127]:
function = lambda *args,**kwags: print(list(args))
xr.apply_ufunc(function,a,b,c,d,e,f)

[array([[275.5, 260. ],
       [325. , 251. ]], dtype=float32), array([[181. , 175.5],
       [205. , 168.5]], dtype=float32), array([[213.5, 196.5],
       [251. , 196.5]], dtype=float32), array([[377. , 372. ],
       [443. , 367.5]], dtype=float32), array([[215., 197.],
       [254., 189.]], dtype=float32), array([[213.5, 209.5],
       [250. , 202.5]], dtype=float32)]


In [31]:
from keras.models import load_model

def get_model():
    global model
    model = load_model("my_model.h5")
    model._make_predict_function()

In [67]:
from keras.models import load_model
model = load_model('my_model.h5')

# # graph = tf.get_default_graph()
# def get_model():
#     global model
#     model = load_model("model_new.h5")
#     model._make_predict_function()
#     print("Model Loaded")


def predict(instance):
#     the_model = kwargs['a_model']
#     print(instance)
    # global graph
#     with graph.as_default():
#     get_model()
    proba = model.predict([[ instance ]])
    index = np.argmax(proba)
    classes = ['agua','bosque','agua_o_bosque','ninguno']
    classes = [1,2,3,4]
    return classes[index]
#     return 1

# %time  get_model()

def j(instance):
    print(instance)
    return instance
    

In [68]:
future = da.apply_along_axis(j, 0, data,dtype=np.double, shape=(2,))
future.compute()

[1 1]
[2 2]
[3 3]
[4 4]
[5 5]
[6 6]


array([[[[1, 2],
         [3, 4],
         [5, 6]],

        [[1, 2],
         [3, 4],
         [5, 6]]]])

In [24]:
future = da.apply_along_axis(sum, 0, data_1,dtype=np.int16, shape=(1,))
future.compute()

IndexError: tuple index out of range

In [114]:
function = lambda *args,**kwags: 
xr.apply_ufunc(function, inv[0], inv[1],inv[2],inv[3],inv[4])

<function sum(iterable, start=0, /)>

In [129]:
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
import keras


data = pd.read_csv('./satellite_dataset.csv')
data.head()

dataset_X = data.drop(columns=['wofs','bosque','wofs_bosque','ninguno'], axis=1)
dataset_X.head()

dataset_Y = data[['wofs','bosque','wofs_bosque','ninguno']]
dataset_Y.head()

X_train, X_test, y_train, y_test = train_test_split(dataset_X, dataset_Y, test_size=0.2)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

# DEFINICIÓN DEL MODELO
# Hay dos maneras de construir modelos en Keras: Secuencial y Funcional
# El modelo secuencial permite construir modelos capa por capa.
# El modelo funcional permite construir modelos mas complicados.

# La capa Flaten transforma los datos de un arreglo bidimensional de 28x28 a un arreglo 
# unidimensional de 784 posiciones(28x28) esto solo formatea el conjutno de datos

# La capa Dense significa que cada neurona en una capa esta conectada a todas las neuronas 
# localizadas en la capa anterior y con todas en la siguiente capa.

model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(6,)),
    keras.layers.Dense(8,input_dim=6, activation='relu'),
    keras.layers.Dense(4, activation='softmax')
])

# CONFIGURACIÓN DE PARÁMETROS
# La compilación del modelo comprende la configuración de parámetros 
# usados durante el entrenamiento: algori tmo de optimización, medida 
# de exactitud, etc.

# optimizer: Adam es un algoitmo de optimización. Además es un método de tasa de aprendizaje adaptativo, 
# loss: calcula la diferencia entre la salida y la variable objetivo. Mide la precisión del modelo durante 
# el entrenamiento y queremos minimizar esta función.
# metrics: Son las métricas que se desan calcular durante el entrenamiento. mide la fracción de imágenes que 
# están clasificadas correctamente.

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen de parámetros del modelo
model.summary()

# ENTRENAMIENTO
model.fit(X_train, y_train, epochs=1)

# EVALUACIÓN
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 56        
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 36        
Total params: 92
Trainable params: 92
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1
800000/800000 [==============================] - 37s 46us/step - loss: 0.3378 - accuracy: 0.9674


ValueError: Tensor Tensor("Mean_274:0", shape=(), dtype=float32) is not an element of this graph.

In [ ]:
future = da.apply_along_axis(predict, 0, data_1,dtype=np.int16, shape=(1,),a_model=model)
future.compute()

In [ ]:
# model._make_predict_function()